Crm Token

In [4]:
import pandas as pd
import json
import requests
import time
import threading
import pyodbc
from concurrent.futures import ThreadPoolExecutor
import ast
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import os
from datetime import datetime, date

'''
ask token
'''    
url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
payload = {
    "grant_type": "password",
    "client_id": "351fb11916fc85504ba5f6ec3b96988b",
    "client_secret": "8f9355878504005ba782ca049d013d4e",
    "redirect_uri": "https://crm-p10.xiaoshouyi.com",
    "username": "11020964@twkd.com",
    "password": "Kd0627ubGHagpQ"
}

response = requests.post(url, data=payload)
content = response.json()
ac_token = content["access_token"]

print(ac_token)

7decfe1399e4d505cdc48b7bdebf05bd72dc8c3780f8d348e63283c165c2999a.MjU0NDU2OTY4NDAzNzYxMA==9


有效客戶

In [5]:

date_0 = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
date_0_scrm = int(date_0.timestamp() * 1000)

'''
select CompanyContact data
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
comp_cont = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
            SELECT id,
                   contactCode__c__c AS contact_code,
                   customItem2__c.contactName AS displayname,
                   name AS ContactID,
                   contactPhone__c__c AS MobilePhone,
                   customItem5__c AS CompanyTypeID,
                   customItem1__c AS customItem8__c,
                   customItem8__c AS CompanyID,
                   dimDepart,
                   ownerId,
                   customItem1__c.accountName AS CoFullName,
                   customItem9__c AS CoShortName,
                   customItem24__c AS employed,
                   customItem109__c AS 勿擾,
                   customItem37__c AS DataRegion,
                   customItem38__c AS 拜電訪標籤,
                   customItem106__c AS LastWorkDescription,
                   customItem103__c AS LastWorkDate,
                   customItem42__c AS Invalid,
                   customItem83__c AS 無效電訪,
                   customItem95__c AS JobTypeID,
                   customItem1__c.SAP_CompanyID__c AS SAP_CompanyID,
                   customItem1__c.Phone__c AS Company_Phone,
                   customItem1__c.customItem227__c AS CompanyID_old,
                   customItem1__c.customItem278__c AS Inactive,
                   customItem50__c as 空號,
                   customItem51__c as 停機,
                   customItem1__c.region__c.customItem9__c as 城市,
                   customItem30__c as 公司地址
            FROM customEntity22__c
            WHERE customItem37__c NOT LIKE 'TW%' AND customItem37__c NOT LIKE 'KDED%'
            AND (customItem5__c LIKE '%C%'
            OR customItem5__c LIKE '%D%')
            ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    comp_cont = pd.concat([comp_cont, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
comp_cont = comp_cont[~comp_cont['CoFullName'].str.contains('Dup>')]
print(comp_cont['DataRegion'].unique())
 
# JobtypeID
comp_cont['JobTypeID'] = comp_cont['JobTypeID'].str.get(0).str[:3]
comp_cont['JobTypeID'] = comp_cont['JobTypeID'].fillna('')
comp_cont = comp_cont[comp_cont['JobTypeID'].isin(["001","002","003","004","005","006","007","009", "010","011","015","017", ""])]

# Contact Mobile
comp_cont['MobilePhone'] = comp_cont['MobilePhone'].str.replace(r'\D', '', regex=True)
comp_cont.loc[comp_cont['MobilePhone'].str.contains("000000"), 'MobilePhone'] = ""
comp_cont.loc[comp_cont['MobilePhone'].str.len() < 6, 'MobilePhone'] = ""

# Company Phone
comp_cont['Company_Phone'] = comp_cont['Company_Phone'].str.replace(r'\D', '', regex=True)
comp_cont.loc[comp_cont['Company_Phone'].str.len() < 7, 'Company_Phone'] = ""

# Co Name
wordlist = "Closed|搬遷|倒閉|歇業|停業|轉行|退休|過世|廢止|解散|燈箱|群組|支援|留守|教育訓練|無效拜訪|資料不全"
comp_cont = comp_cont[~comp_cont['CoFullName'].str.contains(wordlist)]
comp_cont = comp_cont[~comp_cont['CoShortName'].str.contains(wordlist)]

# employed
comp_cont.sort_values(by='id', ascending=False, inplace=True)
comp_cont.drop_duplicates(subset='contact_code', keep='first', inplace=True)
comp_cont['employed'] = comp_cont['employed'].str.get(0)
comp_cont.loc[comp_cont['employed'].str.contains("離職|离职", regex=True, na=False), 'Company_Phone'] = ""

# Invalid
comp_cont['Invalid'] = comp_cont['Invalid'].str.get(0)
comp_cont['Inactive'] = comp_cont['Inactive'].str.get(0)
comp_cont['Invalid'] = comp_cont['Invalid'].fillna("否")
comp_cont['停機'] = comp_cont['停機'].fillna("0")
comp_cont['空號'] = comp_cont['空號'].fillna("0")
comp_cont['Inactive'] = comp_cont['Inactive'].fillna("否")
comp_cont.loc[(comp_cont['Invalid'] == "是") | (comp_cont['停機'] == "1") | (comp_cont['空號'] == "1"), 'MobilePhone'] = ""
comp_cont.loc[comp_cont['Inactive'] == "是", 'Company_Phone'] = ""

# SAP credit
# comp_cont.loc[comp_cont['SAP_CompanyID'].isin(credit_data[credit_data['SAP信用管制'] == "Restrain"]['SAP公司代號']), 'Company_Phone'] = ""

# 勿擾
comp_cont['勿擾'] = comp_cont['勿擾'].fillna('')
comp_cont['勿擾'] = comp_cont['勿擾'].astype(str)
comp_cont = comp_cont[~comp_cont['勿擾'].str.contains("勿電訪|勿傳送簡訊")]

# Remove blank mobile/phone and remove duplicate
comp_cont = comp_cont[~((comp_cont['MobilePhone'] == "") & (comp_cont['Company_Phone'] == ""))]
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].replace('', np.nan)
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].astype(float)
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].dt.tz_convert('Asia/Taipei')
comp_cont['LastWorkDate'] = comp_cont['LastWorkDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
comp_cont_mobile = comp_cont[comp_cont['MobilePhone'] != ""]
comp_cont_no_mobile = comp_cont[comp_cont['MobilePhone'] == ""]

comp_cont_mobile = comp_cont_mobile.sort_values(by='LastWorkDate', ascending=False)
comp_cont_mobile = comp_cont_mobile.drop_duplicates(subset=['MobilePhone'])
comp_cont_no_mobile = comp_cont_no_mobile.sort_values(by='LastWorkDate', ascending=False)
comp_cont_no_mobile = comp_cont_no_mobile.drop_duplicates(subset=['displayname', 'Company_Phone'])

tab = comp_cont_no_mobile['Company_Phone'].value_counts().reset_index()
tab.columns = ['Company_Phone', 'Freq']
comp_cont_no_mobile_nodup = comp_cont_no_mobile[comp_cont_no_mobile['Company_Phone'].isin(tab[tab['Freq'] < 2]['Company_Phone'])]
comp_cont_no_mobile_dup = comp_cont_no_mobile[~comp_cont_no_mobile['Company_Phone'].isin(tab[tab['Freq'] < 2]['Company_Phone'])]


comp_cont_final = pd.concat([comp_cont_mobile, comp_cont_no_mobile_nodup])

####################################主營城市###############################################

#pivot = comp_cont_final.groupby('DataRegion').size().reset_index(name='case_count')
comp_cont_final['城市'] = comp_cont_final['城市'].str.get(0)
comp_cont_final = comp_cont_final[comp_cont_final['城市']=='是']



['SG-INV' 'SG-A2' 'SG-A1' 'MY-A1' 'SG-N1' 'SG-A6' '印度分公司' 'US-East'
 'HK-INV' 'MY-A2' 'HK-E' 'HK-A' 'HK-D' 'HK-C' 'MY-INV' 'ID-A' 'ID-D'
 'ID-B' 'PH-A2' 'TH-A3' 'VN-A1' 'TH-A2' 'ID-C' 'TH-A1' 'PH-A1' 'PD'
 'TH-INV' 'VN-INV' 'MY-A3' 'PH-INV' 'ID-INV' 'CAN' 'ATND(無效資料)' 'US-West'
 'PH-N1' 'MY-N1' 'TH-N1' 'DE' 'TH-Others' 'JP' 'US-Central' 'AU' 'US-INV'
 'KR' '台灣-海外' 'Intl Dept.' 'United States Branch' '資訊部' '數據中心課' 'VN-N1'
 'ID-N1' '香港分公司' 'HK-N1' 'Thailand Branch' 'NZ' '績效組' 'PH-加盟' 'AU-加盟'
 'VN-加盟' 'Singapore Branch' '(開發)北區' 'Japan Branch' 'AU-雪梨']


In [ ]:
'''
obtain table type to get the entityType
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/xobjects/customEntity47__c/busiType"
headers = {
    "Authorization": f"Bearer {ac_token}"
    #"Content-Type":"application/json"    #json data type
}
response2 = requests.get(url_2, headers = headers)
content2 = response2.json()
print(content2)
entityType_df = pd.json_normalize(content2["data"]["records"]) 

'''
complete K on new crm K大預約表
'''
date_60 = datetime.now() - timedelta(days=60)
date_60_scrm = int(date_60.timestamp() * 1000)

url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
OS_K_SCRM = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''SELECT id, name
                    ,customItem30__c
                    ,customItem6__c.name
                    ,customItem18__c as DataRegion
                    ,customItem38__c as reserved_date
                    ,customItem3__c.customItem322__c as 目標客戶類型
                    FROM customEntity23__c
                    WHERE entityType = '3095935350757778'
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    OS_K_SCRM = pd.concat([OS_K_SCRM, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
OS_K_SCRM['customItem30__c'] = OS_K_SCRM['customItem30__c'].str.get(0)
OS_K_SCRM = OS_K_SCRM[(OS_K_SCRM['customItem30__c'] == "是")]
OS_K_SCRM['目標客戶類型'] = OS_K_SCRM['目標客戶類型'].str.get(0)
#comp_cont_final = comp_cont_final[~comp_cont_final['ContactID_old'].isin(OS_K_CRM_Done['ContactID'])]
OS_K_SCRM['reserved_date'] = pd.to_numeric(OS_K_SCRM['reserved_date'], errors='coerce') 
OS_K_SCRM['reserved_date'] = OS_K_SCRM['reserved_date'].astype(float)
OS_K_SCRM['reserved_date'] = OS_K_SCRM['reserved_date'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
OS_K_SCRM['reserved_date'] = OS_K_SCRM['reserved_date'].dt.tz_convert('Asia/Taipei')
OS_K_SCRM['reserved_date'] = OS_K_SCRM['reserved_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
OS_K_SCRM = OS_K_SCRM[OS_K_SCRM['customItem6__c.name']!='']
OS_K_SCRM['DataRegion'] = OS_K_SCRM['DataRegion'].str[:2]
OS_K_SCRM['reserved_date'] = pd.to_datetime(OS_K_SCRM['reserved_date'])
over_two = OS_K_SCRM[OS_K_SCRM['reserved_date'] < date_60]
OS_K_SCRM_in_two = OS_K_SCRM[~OS_K_SCRM['name'].isin(over_two['name'])]
comp_cont_final = comp_cont_final[~comp_cont_final['ContactID'].isin(OS_K_SCRM_in_two['customItem6__c.name'])]
comp_cont_final.loc[comp_cont_final['ContactID'].isin(over_two['customItem6__c.name']), 'customItem2__c'] = 'Daily Invite Mr.K- Ver. 2'

'''
新加坡做過K大且買過美耐板經營客戶排除(from Tia)
發放申請
新加坡分公司 = 7
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
already_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
                   select id, accountCode__c, customItem25__c.name as ContactID,
                   customItem72__c as 領取地點
                   from customEntity25__c
                   where customItem72__c =7
                   
                ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    already_df = pd.concat([already_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
    
#發放明細   
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
detail_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
                   select catalogGiftSendRequest__c, gift__c.name as 物品名稱,
                   customItem61__c AS 國家,
                   createDate__c as 建檔日期
                   from customEntity28__c
                   where customItem49__c LIKE 'SG%' or customItem49__c = 'Singapore'

                ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    detail_df = pd.concat([detail_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

already_df = pd.merge(already_df, detail_df, left_on = 'id', right_on = 'catalogGiftSendRequest__c', how = 'left')
already_df = already_df[~already_df['物品名稱'].isna()]
already_df = already_df[already_df['物品名稱'].str.startswith('L')]

sg_done = OS_K_SCRM[(OS_K_SCRM['目標客戶類型'] == '經營客戶') & (OS_K_SCRM['DataRegion'] == 'SG')]
sg_done = pd.merge(sg_done, already_df, left_on = 'customItem6__c.name', right_on = 'ContactID', how = 'left')
sg_done = sg_done[~sg_done['物品名稱'].isna()]
comp_cont_final = comp_cont_final[~comp_cont_final['ContactID'].isin(sg_done['customItem6__c.name'])]
# sg_done.to_excel('sg_done.xlsx' , index=False)
'''
K大餐會人
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
K_visit_df1 = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''select name, customItem2__c.name as ContactID, customItem1__c.customItem18__c as DataRegion, customItem1__c.customItem2__c as reserved_date,
                          customItem1__c.customItem30__c as 是否舉行
        from customEntity24__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    K_visit_df1 = pd.concat([K_visit_df1, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
K_visit_df1['DataRegion'] = K_visit_df1['DataRegion'].str[:2]
K_visit_df1 = K_visit_df1[K_visit_df1['DataRegion']!='TW']
K_visit_df1['reserved_date'] = pd.to_numeric(K_visit_df1['reserved_date'], errors='coerce') 
K_visit_df1['reserved_date'] = K_visit_df1['reserved_date'].astype(float)
K_visit_df1['reserved_date'] = K_visit_df1['reserved_date'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
K_visit_df1['reserved_date'] = K_visit_df1['reserved_date'].dt.tz_convert('Asia/Taipei')
K_visit_df1['reserved_date'] = K_visit_df1['reserved_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
K_visit_df1['reserved_date'] = pd.to_datetime(K_visit_df1['reserved_date'])
K_visit_df1['是否舉行'] = K_visit_df1['是否舉行'].str.get(0)
K_visit_df1 = K_visit_df1[(K_visit_df1['是否舉行'] == "是")]
over_two = K_visit_df1[K_visit_df1['reserved_date'] < date_60]
K_visit_df1_in_two = K_visit_df1[~K_visit_df1['name'].isin(over_two['name'])]
comp_cont_final = comp_cont_final[~comp_cont_final['ContactID'].isin(K_visit_df1_in_two['ContactID'])]
comp_cont_final.loc[comp_cont_final['ContactID'].isin(over_two['ContactID']), 'customItem2__c'] = 'Daily Invite Mr.K- Ver. 2'

#comp_cont_final = comp_cont_final[~comp_cont_final['ContactID'].isin(K_visit_df1['ContactID'])]
#pivot3 = comp_cont_final.groupby('DataRegion').size().reset_index(name='case_count')


'''
remove those complete visit in three months in SG
'''
date_6m = datetime.now() - timedelta(days=6*30)
date_6m_scrm = int(date_6m.timestamp() * 1000)

url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
TWOS_tracking = pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT id
                        ,name
                        ,accountCode__c
                        ,customItem40__c
                        ,customItem168__c
                        ,customItem128__c
                        ,customItem48__c.name
                        ,dimDepart.departName
                            FROM customEntity15__c
                        WHERE dimDepart.departName LIKE 'SG%'
                        AND customItem40__c >= {date_6m_scrm}
                        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    TWOS_tracking = pd.concat([TWOS_tracking, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
TWOS_tracking['customItem128__c'] = TWOS_tracking['customItem128__c'].str.get(0)
TWOS_tracking['customItem40__c'] = TWOS_tracking['customItem40__c'].replace('', np.nan)
TWOS_tracking['customItem40__c'] = TWOS_tracking['customItem40__c'].astype(float)
TWOS_tracking['customItem40__c'] = TWOS_tracking['customItem40__c'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
TWOS_tracking['customItem40__c'] = TWOS_tracking['customItem40__c'].dt.tz_convert('Asia/Taipei')
TWOS_tracking['customItem40__c'] = TWOS_tracking['customItem40__c'].dt.strftime('%Y-%m-%d %H:%M:%S')
TWOS_tracking = TWOS_tracking[(TWOS_tracking['customItem168__c'].isin(['A2', 'A2-8', 'A4'])) | (TWOS_tracking['customItem128__c'] == "A1 拜訪")]
comp_cont_final = comp_cont_final[~comp_cont_final['ContactID'].isin(TWOS_tracking['customItem48__c.name'])]
#pivot4 = comp_cont_final.groupby('DataRegion').size().reset_index(name='case_count')


'''
special exclude
'''

comp_cont_final = comp_cont_final[~comp_cont_final['ContactID'].isin(["KHLXRGX20231102114151","KHLXRGX2023110298720","KHLXRGX2023110228629", "KHLXRGX2023110221895", "KHLXRGX202310042112", "KHLXRGX2023110234140", "KHLXRGX2023110225511", "KHLXRGX2023110293730"])]
comp_cont_final = comp_cont_final[~comp_cont_final['CompanyID'].isin(["GAC00096758", "GAC00075392", "GAC00071671", "GAC00074439", "GAC00096845", "GAC00067286", "GAC00096812"])]
#aaa = comp_cont_final[comp_cont_final['DataRegion'] == "MY"]
#comp_cont_final = comp_cont_final[comp_cont_final['DataRegion'] != "MY"]
comp_cont_final = comp_cont_final.sort_values(by='LastWorkDate')
comp_cont_final = comp_cont_final.drop_duplicates(subset=['CompanyID'])
comp_cont_final['Operator'] = pd.NA
comp_cont_final['EmplID'] = pd.NA

'''
execute from scrm
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
calling_record_SCRM = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''SELECT id
                            ,name
                            ,customItem14__c
                            ,customItem14__c.name
                            ,customItem14__c.customItem98__c
                            ,customItem22__c
                            ,customItem38__c
                            ,approvalStatus
                            ,customItem7__c
                            ,customItem39__c
                            from customEntity47__c
                            where entityType = '3093637455518085'
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    calling_record_SCRM = pd.concat([calling_record_SCRM, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
calling_record_SCRM = calling_record_SCRM[calling_record_SCRM['customItem14__c']!='']
calling_record_SCRM['customItem38__c'] = calling_record_SCRM['customItem38__c'].str.get(0)
calling_record_SCRM['customItem22__c'] = calling_record_SCRM['customItem22__c'].str.get(0)
calling_record_SCRM['approvalStatus'] = calling_record_SCRM['approvalStatus'].str.get(0)
calling_record_SCRM = calling_record_SCRM[(calling_record_SCRM['customItem22__c'] == "任務完成")]
calling_record_SCRM['customItem7__c'] = calling_record_SCRM['customItem7__c'].replace('', np.nan)
calling_record_SCRM['customItem7__c'] = calling_record_SCRM['customItem7__c'].astype(float)
calling_record_SCRM['customItem7__c'] = calling_record_SCRM['customItem7__c'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
calling_record_SCRM['customItem7__c'] = calling_record_SCRM['customItem7__c'].dt.tz_convert('Asia/Taipei')
calling_record_SCRM['customItem7__c'] = calling_record_SCRM['customItem7__c'].dt.strftime('%Y-%m-%d %H:%M:%S')
calling_record_SCRM = calling_record_SCRM.sort_values(by='customItem7__c', ascending=True)
calling_record_SCRM_pick = calling_record_SCRM[['customItem14__c', 'customItem7__c', 'customItem38__c', 'customItem39__c']]
calling_record_SCRM_pick.drop_duplicates(subset=['customItem14__c'], inplace=True)
calling_record_SCRM_pick['customItem39__c'] = calling_record_SCRM_pick['customItem39__c'].str.get(0)
calling_record_SCRM_pick = calling_record_SCRM_pick[(calling_record_SCRM_pick['customItem39__c'] != '成功邀約K大') & (calling_record_SCRM_pick['customItem39__c'] != '已完成過K大')]
#calling_record_SCRM_pick = calling_record_SCRM_pick[(calling_record_SCRM_pick['customItem38__c'] != "未接")]
#calling_record_SCRM_nopick = calling_record_SCRM[(calling_record_SCRM['customItem38__c'] == "未接")]
#calling_record_SCRM_pick = calling_record_SCRM[(calling_record_SCRM['customItem22__c'] == "任務完成")]##############################################
#comp_cont_final = comp_cont_final[~comp_cont_final['ContactID'].isin(calling_record_SCRM_pick['customItem14__c.name'])]

'''
customItem39__c != 1 不等於邀約K大, customItem38__c = 3 接通掛電話
'''
date_7 = datetime.now() - timedelta(days=7)
date_7_scrm = int(date_7.timestamp() * 1000)

date_3 = datetime.now() - timedelta(days=3)
date_3_scrm = int(date_3.timestamp() * 1000)



url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
exist_SCRM = pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT 
                            id
                            ,name
                            ,customItem14__c
                            ,customItem7__c
                            ,customItem38__c
                            from customEntity47__c
                            where 
                            (customItem39__c != 1 or customItem38__c = 3)and customItem7__c >= {date_7_scrm}
                            
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    exist_SCRM = pd.concat([exist_SCRM, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
comp_cont_final = comp_cont_final[~comp_cont_final['id'].isin(exist_SCRM['customItem14__c'])]


date_2 = datetime.now() - timedelta(days=2)
date_2_scrm = int(date_2.timestamp() * 1000)


'''
execute from scrm 未接兩天內不重複
customItem38__c =1 未接
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
nopick_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT 
                            id
                            ,name
                            ,customItem14__c
                            ,customItem7__c
                            ,customItem38__c
                            from customEntity47__c
                            where 
                            customItem38__c = 1 and customItem7__c >= {date_2_scrm}
                            
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    nopick_df = pd.concat([nopick_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
comp_cont_final = comp_cont_final[~comp_cont_final['id'].isin(nopick_df['customItem14__c'])]
#pivot7 = comp_cont_final.groupby('DataRegion').size().reset_index(name='case_count')


# '''
# 不和今天重複
# '''
# url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
# headers = {
#     "Authorization": f"Bearer {ac_token}",
#     "Content-Type":"application/x-www-form-urlencoded"
#     # Replace with your actual access token
# }
# queryLocator = ''
# today_SCRM = pd.DataFrame()
 
# while True:
#     data = {
#         "xoql": f'''SELECT 
#                             id
#                             ,name
#                             ,customItem14__c
#                             ,customItem7__c
#                             ,customItem38__c
#                             from customEntity47__c
#                             where 
#                             createdBy = '2988448346347872'
#                             and customItem7__c >= {date_0_scrm}
                            
#                    ''',
#         "batchCount": 2000,
#         "queryLocator": queryLocator
#     }
#     response = requests.post(url_2, headers=headers, data=data)
#     crm = response.json()
#     data = pd.DataFrame(crm["data"]["records"])
#     today_SCRM = pd.concat([today_SCRM, data], ignore_index=True, sort=False)
    
#     if not crm['queryLocator']:
#         break
#     queryLocator = crm['queryLocator']
# comp_cont_final = comp_cont_final[~comp_cont_final['id'].isin(today_SCRM['customItem14__c'])]

'''
去除有下一次聯絡時間
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
next_contact = pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT 
                            id
                            ,name
                            ,customItem14__c
                            ,customItem27__c as 下次聯絡日期
                            ,customItem38__c
                            from customEntity47__c
                            where customItem27__c >= {date_0_scrm}
                            
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    next_contact = pd.concat([next_contact, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
comp_cont_final = comp_cont_final[~comp_cont_final['id'].isin(next_contact['customItem14__c'])]
#pivot8 = comp_cont_final.groupby('DataRegion').size().reset_index(name='case_count')


#檳城
#comp_cont_final = comp_cont_final[comp_cont_final['dimDepart']=='3112866870415255']

'''
去追蹤紀錄
customItem128__c = 3 B1電訪, customItem4__c = 4 K大邀約, customItem112__c = 1 成功邀約K大
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
tr_df = pd.DataFrame()
 #成功邀約K大
while True:
    data = {
        "xoql": '''SELECT customItem48__c, customItem118__c as DataRegion
                        , customItem113__c as reserved_date
                        , name
                        , customItem112__c as 是否邀約K大
                        from customEntity15__c
                        where customItem128__c = 3 and customItem4__c = 4  
                        and entityType = '3028403761699212' 
                        and customItem112__c = 1   
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    tr_df = pd.concat([tr_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
tr_df['DataRegion'] = tr_df['DataRegion'].str[:2]
tr_df = tr_df[tr_df['DataRegion']!='TW']
tr_df['reserved_date'] = pd.to_numeric(tr_df['reserved_date'], errors='coerce') 
tr_df['reserved_date'] = tr_df['reserved_date'].astype(float)
tr_df['reserved_date'] = tr_df['reserved_date'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
tr_df['reserved_date'] = tr_df['reserved_date'].dt.tz_convert('Asia/Taipei')
tr_df['reserved_date'] = tr_df['reserved_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
tr_df = tr_df[tr_df['customItem48__c']!='']
tr_df = tr_df[~tr_df['reserved_date'].isna()]
tr_df['reserved_date'] = pd.to_datetime(tr_df['reserved_date'])
over_two = tr_df[tr_df['reserved_date'] < date_60]
tr_df_in_two = tr_df[~tr_df['name'].isin(over_two['name'])]
comp_cont_final = comp_cont_final[~comp_cont_final['id'].isin(tr_df_in_two['customItem48__c'])]
comp_cont_final.loc[comp_cont_final['id'].isin(over_two['customItem48__c']), 'customItem2__c'] = 'Daily Invite Mr.K- Ver. 2'

#comp_cont_final = comp_cont_final[~comp_cont_final['id'].isin(tr_df['customItem48__c'])]
#pivot10 = comp_cont_final.groupby('DataRegion').size().reset_index(name='case_count')



'''
處理之前有過任務完成但沒有K大過
'''
oversea_df = pd.merge(comp_cont_final, calling_record_SCRM_pick, left_on = 'id', right_on = 'customItem14__c', how = 'left')
oversea_df['nopick'] = False
# Update nopick column based on condition
oversea_df.loc[oversea_df['customItem38__c']=='未接', 'nopick'] = True
oversea_df['customItem4__c'] = np.where(oversea_df['nopick'], '2', '1')
final_1 = oversea_df[oversea_df['customItem14__c'].isna()]
final_2 = oversea_df[oversea_df['customItem38__c'] == '未接']
final_3 = oversea_df[~oversea_df['id'].isin(final_1['id']) & ~oversea_df['id'].isin(final_2['id'])]

def process_data_region(region):
    if region.startswith('KDED'):
        return region
    else:
        return region[:2]
final_1['DataRegion'] = final_1['DataRegion'].apply(process_data_region)
final_2['DataRegion'] = final_2['DataRegion'].apply(process_data_region)
final_3['DataRegion'] = final_3['DataRegion'].apply(process_data_region)
#grouped_counts1 = final_1.groupby('DataRegion').size()
#grouped_counts2 = final_2.groupby('DataRegion').size()
#grouped_counts3 = final_3.groupby('DataRegion').size()
final_3 = final_3.sort_values(by='customItem7__c', ascending=True)
#pivot11 = final_3.groupby('DataRegion').size().reset_index(name='case_count')
first_nopick = pd.concat([final_1, final_2], ignore_index=True)
#pivot12 = first_nopick.groupby('DataRegion').size().reset_index(name='case_count')
